# Project 10 -- Anthony Micco

**TA Help:** N/A

**Collaboration:** N/A

**Internet Resources:** N/A

**ChatGPT, Gemini, Claude, etc:** NONE

***Link to AI Chat History***: NONE \
**OVERALL MESSAGE:** Any time that you used anything except your brain to solve the questions in these projects, you need to disclose such resources at the start of the project, with details about your usage of the tools.

**YOUR OWN WORK:** Even when you utilize other resources, do NOT just copy and paste.  Write all explanations in your own words, using several sentences in English, which are understandable and which you wrote (and did not just copy and paste).

## Question 1

In [13]:
# connecting to a MongoDB instance using Python's pymongo library

from pymongo import MongoClient
import pandas as pd
import json

# create a client to connect to the local MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

# Create or connect to a database
db = client['movies_db']

# Create or connect to a collection
movies = db['movies']

In [14]:
#importing the movies data into the MongoDB using the client we made in the previous cell

# Read the CSV file and convert to MongoDB documents
df = pd.read_csv('/anvil/projects/tdm/data/MovieLens/movies_metadata.csv')

# Convert DataFrame to list of dictionaries
movies_data = df.to_dict('records')

# Insert the data into MongoDB
result = movies.insert_many(movies_data)
print(f"Inserted {len(result.inserted_ids)} movies")

/tmp/ipykernel_216/622833.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/anvil/projects/tdm/data/MovieLens/movies_metadata.csv')


Inserted 45466 movies


In [15]:
#viewing the first 5 rows of data from the movies dataset

# Find all movies (limit to first 5 for display)
sample_movies = list(movies.find().limit(5))
print(f"Total movies in collection: {movies.count_documents({})}")
print("\nSample movie documents:")
for movie in sample_movies:
    print(f"Title: {movie.get('title', 'N/A')}")
    print(f"Release Date: {movie.get('release_date', 'N/A')}")
    print(f"Vote Average: {movie.get('vote_average', 'N/A')}")
    print("---")

Total movies in collection: 136398

Sample movie documents:
Title: Toy Story
Release Date: 1995-10-30
Vote Average: 7.7
---
Title: Jumanji
Release Date: 1995-12-15
Vote Average: 6.9
---
Title: Grumpier Old Men
Release Date: 1995-12-22
Vote Average: 6.5
---
Title: Waiting to Exhale
Release Date: 1995-12-22
Vote Average: 6.1
---
Title: Father of the Bride Part II
Release Date: 1995-02-10
Vote Average: 5.7
---


In [16]:
#viewing the structure of each movie document or record

# Get the structure of a movie document
sample_movie = movies.find_one()
print("Movie document structure:")
for key, value in sample_movie.items():
    print(f"{key}: {type(value).__name__}")
    if isinstance(value, str) and len(str(value)) > 100:
        print(f"  Value: {str(value)[:100]}...")
    else:
        print(f"  Value: {value}")
    print()

Movie document structure:
_id: ObjectId
  Value: 690109e45817d5eff963ff2f

adult: str
  Value: False

belongs_to_collection: str
  Value: {'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'ba...

budget: str
  Value: 30000000

genres: str
  Value: [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

homepage: str
  Value: http://toystory.disney.com/toy-story

id: str
  Value: 862

imdb_id: str
  Value: tt0114709

original_language: str
  Value: en

original_title: str
  Value: Toy Story

overview: str
  Value: Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto ...

popularity: float
  Value: 21.946943

poster_path: str
  Value: /rhIRbceoE9lR4veEXuwCC2wARtG.jpg

production_companies: str
  Value: [{'name': 'Pixar Animation Studios', 'id': 3}]

production_countries: str
  Value: [{'iso_3166_1': 'US', 'name': 'United States of America'}]

release_dat

1.1) There are 45466 movies in the collection \
1.2) The structure of a movie document contains 25 different columns \
1.3) There are a variety of different fields available in each movie document such as runtime, title, video, and adult among others \
1.4) Some sample movie titles include Toy Story, Jumanji, and Grumpier Old Men

## Question 2

In [21]:
# using practice query to find all movies with a rating above 8.0 

# Find movies with rating above 8.0
high_rated_movies = list(movies.find({"vote_average": {"$gt": 8.0}}))
print(f"Movies with rating above 8.0: {len(high_rated_movies)}")
for movie in high_rated_movies[:5]:  # Show first 5
    print(f"{movie['title']} - Rating: {movie['vote_average']}")

Movies with rating above 8.0: 3390
Se7en - Rating: 8.1
The Usual Suspects - Rating: 8.1
Taxi Driver - Rating: 8.1
Reckless - Rating: 10.0
Star Wars - Rating: 8.1


In [22]:
# practicing understanding how to do multiple filters when querying ($and operator)

# Example: Find movies with rating above 7.0 AND released after 2000
recent_good_movies = list(movies.find({
    "$and": [
        {"vote_average": {"$gt": 7.0}},
        {"release_date": {"$regex": "200[0-9]|201[0-9]|202[0-9]"}}  # Regex for years 2000+, as the release year is a string. This is a common pattern for dates in MongoDB.
    ]
}))
print(f"Good movies released after 2000: {len(recent_good_movies)}")
for movie in recent_good_movies[:5]:  # Show first 5
    print(f"{movie['title']} - Rating: {movie['vote_average']}, Year: {movie['release_date'][:4]}")

Good movies released after 2000: 11688
Venice - Rating: 7.5, Year: 2010
The Sleepover - Rating: 8.0, Year: 2013
The Farmer's Wife - Rating: 10.0, Year: 2012
Black Tar Heroin: The Dark End of the Street - Rating: 7.1, Year: 2000
Erin Brockovich - Rating: 7.1, Year: 2000


In [28]:
#writing a query to find movies with a rating above 6.0 and released in the 1990s 

ninetiesMovies = list(movies.find({
    "$and":[ {"vote_average":{"$gt":6.0}},
            {"release_date":{"$regex":"199[0-9]"}}
           ]
}))

print(f"Movies rated higher than 6.0, released in the 1990s: {len(ninetiesMovies)}")
for movie in ninetiesMovies[:5]:
    print(f"{movie['title']} - Rating: {movie['vote_average']}, Year: {movie['release_date'][:4]}")

Movies rated higher than 6.0, released in the 1990s: 7734
Toy Story - Rating: 7.7, Year: 1995
Jumanji - Rating: 6.9, Year: 1995
Grumpier Old Men - Rating: 6.5, Year: 1995
Waiting to Exhale - Rating: 6.1, Year: 1995
Heat - Rating: 7.7, Year: 1995


In [27]:
#writing a query to find movies from the comedy genre

comedies = list(movies.find({
    "genres":{"$regex":"Comedy", "$options":"i"}
}))
print(f"There are {len(comedies)} comedy movies")
for movie in comedies[:5]:
    print(f"{movie['title']} - Genres: {movie.get('genres','N/A')}")

There are 39546 comedy movies
Toy Story - Genres: [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]
Grumpier Old Men - Genres: [{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]
Waiting to Exhale - Genres: [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]
Father of the Bride Part II - Genres: [{'id': 35, 'name': 'Comedy'}]
Sabrina - Genres: [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]


2.4) There are 7734 movies with a rating higher than 6.0 released in the 1990s. There were 39546 movies that were comedies.

## Question 3

In [30]:
# testing out MongoDB's sort method 

# Find all movies sorted by rating (descending)
movies_by_rating = list(movies.find().sort("vote_average", -1))
print("Movies sorted by rating (highest first):")
for movie in movies_by_rating[:5]:  # Show top 5
    print(f"{movie['title']} - Rating: {movie['vote_average']}")

Movies sorted by rating (highest first):
Reckless - Rating: 10.0
Girl in the Cadillac - Rating: 10.0
The Haunted World of Edward D. Wood, Jr. - Rating: 10.0
Carmen Miranda: Bananas Is My Business - Rating: 10.0
Other Voices Other Rooms - Rating: 10.0


In [31]:
#finding the highest rated movie in the dataset

highest_rated = movies.find().sort("vote_average", -1).limit(1)
top_movie = list(highest_rated)[0]
print(f"Highest rated movie: {top_movie['title']} - Rating: {top_movie['vote_average']}")

Highest rated movie: Reckless - Rating: 10.0


In [35]:
#writing a query to find the three most popular movies released after 2000

moviesAfter2000 = list(movies.find({
    "release_date":{"$regex":"200[0-9]|201[0-0]|202[0-9]"}
}).sort("vote_average",-1).limit(3))
for movie in moviesAfter2000: 
    print(f"{movie['title']} - Rating: {movie['vote_average']}, Year: {movie['release_date'][:4]}")

Forever - Rating: 10.0, Year: 2006
Pavement: Slow Century - Rating: 10.0, Year: 2002
High Roller: The Stu Ungar Story - Rating: 10.0, Year: 2003


In [36]:
#practicing using the count_documents() method

total_movies = movies.count_documents({})
high_rated_count = movies.count_documents({"vote_average": {"$gt": 8.0}})
recent_movies_count = movies.count_documents({"release_date": {"$regex": "201[0-9]|202[0-9]"}})

print(f"Total movies: {total_movies}")
print(f"Movies with rating above 8.0: {high_rated_count}")
print(f"Movies released in 2010s or 2020s: {recent_movies_count}")

Total movies: 136398
Movies with rating above 8.0: 3390
Movies released in 2010s or 2020s: 38400


In [40]:
#finding the count of movies with a rating greater than 7 released in the 1990s

moviesInThe90s = movies.count_documents(
    {"$and" :[
        {"vote_average":{"$gt":7.0}}, 
        {"release_date": {"$regex": "199[0-9]"}}
    ]
    })

print(f"Total movies with rating above 7.0 released in the 1990s: {moviesInThe90s}")

Total movies with rating above 7.0 released in the 1990s: 2664


3.4) The three most popular movies released after 2000 are Forever, Pavement: Slow Century, and High Roller: The Stu Ungar Story. There were 2664 movies with a rating greater than 7 released in the 1990s.

## Question 4

In [42]:
# basic aggregation pipeline structure

pipeline = [
    {"$match": {"field": "value"}},  # Filter documents (like WHERE)
    {"$group": {"_id": "$field", "count": {"$sum": 1}}},  # Group and aggregate
    {"$sort": {"count": -1}}  # Sort results
    # ... # other operations can be added here
]

#result = collection.aggregate(pipeline)

In [43]:
#performing a simple aggregation of the movies' genres to understand its functionality

# Count movies by genre (this is a simplified example - real genre data might be more complex)
pipeline = [
    {"$group": {"_id": "$genres", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
]

genre_counts = list(movies.aggregate(pipeline))

print("Movies by genre (top 10):")
for genre in genre_counts:
    print(f"{genre['_id']}: {genre['count']} movies")

Movies by genre (top 10):
[{'id': 18, 'name': 'Drama'}]: 15000 movies
[{'id': 35, 'name': 'Comedy'}]: 10863 movies
[{'id': 99, 'name': 'Documentary'}]: 8169 movies
[]: 7326 movies
[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]: 3903 movies
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}]: 3405 movies
[{'id': 27, 'name': 'Horror'}]: 2922 movies
[{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]: 2790 movies
[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]: 1779 movies
[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}]: 1596 movies


In [44]:
#finding the average rating of movies by their release year utilizing $addFields to extract the year

# Average rating by release year
pipeline = [
    {"$addFields": {
        "year": {"$substr": ["$release_date", 0, 4]}  # Extract year from date
    }},
    {"$match": {"year": {"$ne": ""}}},  # Filter out empty years
    {"$group": {
        "_id": "$year",
        "avg_rating": {"$avg": "$vote_average"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"_id": -1}},  # Sort by year descending
    {"$limit": 10}
]

avg_ratings = list(movies.aggregate(pipeline))
print("Average rating by year (recent years):")
for year in avg_ratings:
    print(f"{year['_id']}: {year['avg_rating']:.2f} rating ({year['count']} movies)")

Average rating by year (recent years):
nan: nan rating (261 movies)
22: nan rating (3 movies)
2020: 0.00 rating (3 movies)
2018: 0.00 rating (15 movies)
2017: 5.86 rating (1596 movies)
2016: 5.82 rating (4812 movies)
2015: 5.76 rating (5715 movies)
2014: 5.65 rating (5922 movies)
2013: 5.76 rating (5667 movies)
2012: 5.64 rating (5166 movies)


In [47]:
#using the $match stage to filter documents before grouping to find the average rating by popularity ranges

pipeline = [
    {"$match": {
        "popularity": {"$exists": True, "$gt": 0}, #making sure popularity score exists and is greater than 0
        "vote_average": {"$gt": 0} #making sure vote_average is greater than 0
    }},
    {"$addFields": {
        "popularity_range": { #creating new field popularity_range based on the value of popularity
            "$switch": {
                "branches": [
                    {"case": {"$lt": ["$popularity", 10]}, "then": "Low (0-10)"},
                    {"case": {"$lt": ["$popularity", 50]}, "then": "Medium (10-50)"},
                    {"case": {"$lt": ["$popularity", 100]}, "then": "High (50-100)"}
                ],
                "default": "Very High (100+)"
            }
        }
    }},
    {"$group": {
        "_id": "$popularity_range", #grouping by the new field
        "avg_rating": {"$avg": "$vote_average"},
        "avg_popularity": {"$avg": "$popularity"},
        "count": {"$sum": 1}
    }},
    {"$sort": {"avg_popularity": 1}} #sorting by the average popularity score
]

popularity_ratings = list(movies.aggregate(pipeline))
print("Average rating by popularity ranges:")
for pop_range in popularity_ratings:
    print(f"{pop_range['_id']}: {pop_range['avg_rating']:.2f} avg rating, {pop_range['avg_popularity']:.1f} avg popularity ({pop_range['count']} movies)")

Average rating by popularity ranges:
Low (0-10): 6.01 avg rating, 2.5 avg popularity (85392 movies)
Medium (10-50): 6.46 avg rating, 13.6 avg popularity (7959 movies)
High (50-100): 7.83 avg rating, 71.4 avg popularity (24 movies)
Very High (100+): 7.35 avg rating, 195.8 avg popularity (36 movies)


4.4) From these aggregation results you can see how columns relate to one another such as year and the average rating as we found. From this, we could potentially analyze if movies are getting better or worse overtime. We can also see that there are not many movies that are highly rated as most movies are rated below 7, whereas there are fewer movies that are highly rated and rated above 7.

## Question 5

In [49]:
# updating documents using the update_one() and update_many() methods

# Update a single document (find a specific movie by title)
result = movies.update_one(
    {"title": "Toy Story"},
    {"$set": {"our_rating": 9.5}}
)
print(f"Updated {result.modified_count} document")

# Check the update
updated_movie = movies.find_one({"title": "Toy Story"})
print(f"Toy Story's our rating: {updated_movie.get('our_rating', 'No rating')}")

Updated 1 document
Toy Story's our rating: 9.5


In [50]:
# Update all movies to add a 'watched' field
result = movies.update_many(
    {},
    {"$set": {"watched": False}}
)
print(f"Updated {result.modified_count} documents with watched field")

Updated 136398 documents with watched field


In [51]:
# deleting documents using the delete_one() and delete_many() methods

# Delete movies with very low ratings (be careful with this!)
result = movies.delete_many({"vote_average": {"$lt": 2.0}})
print(f"Deleted {result.deleted_count} movies with rating below 2.0")

# Check how many movies remain
remaining_movies = movies.count_documents({})
print(f"Remaining movies: {remaining_movies}")

Deleted 9618 movies with rating below 2.0
Remaining movies: 126780


In [52]:
# Delete movies from a specific year (example: 1990)
result = movies.delete_many({"release_date": {"$regex": "1990"}})
print(f"Deleted {result.deleted_count} movies from 1990")

# Check remaining movies
remaining_movies = movies.count_documents({})
print(f"Remaining movies: {remaining_movies}")

# Show some sample remaining movies
sample_movies = list(movies.find().limit(3))
print("Sample remaining movies:")
for movie in sample_movies:
    print(f"- {movie['title']} ({movie.get('release_date', 'N/A')})")

Deleted 1182 movies from 1990
Remaining movies: 125598
Sample remaining movies:
- Toy Story (1995-10-30)
- Jumanji (1995-12-15)
- Grumpier Old Men (1995-12-22)


## Pledge

By submitting this work I hereby pledge that this is my own, personal work. I've acknowledged in the designated place at the top of this file all sources that I used to complete said work, including but not limited to: online resources, books, and electronic communications. I've noted all collaboration with fellow students and/or TA's. I did not copy or plagiarize another's work.

> As a Boilermaker pursuing academic excellence, I pledge to be honest and true in all that I do. Accountable together – We are Purdue.

https://www.purdue.edu/odos/osrr/honor-pledge/
